In [1]:
import pennylane as qml

In [2]:
import torch
import numpy as np

In [3]:
qpu = qml.device('qiskit.ibmq', wires=2, backend='ibmq_vigo',shots=100)
#qpu = qml.device('strawberryfields.fock', wires=2, cutoff_dim=10)
#qpu = qml.device('strawberryfields.gaussian', wires=2)
#qpu = qml.device('qiskit.aer', wires=2)

In [4]:
qpu.capabilities()

{'model': 'qubit',
 'tensor_observables': True,
 'backend': ['ibmq_qasm_simulator',
  'ibmqx2',
  'ibmq_16_melbourne',
  'ibmq_vigo',
  'ibmq_ourense',
  'ibmq_london',
  'ibmq_burlington',
  'ibmq_essex']}

In [5]:
phi=torch.tensor(0.0)

In [6]:
phi

tensor(0.)

In [7]:
phi.requires_grad=True

In [8]:
theta=torch.tensor(1e-5)

In [9]:
theta.requires_grad=True

In [10]:
target=-1

In [11]:
@qml.qnode(qpu,interface='torch')
def circuit(phi,theta):
    #qml.BasisState(np.asarray([0,0]),wires=[0,1])
    qml.RX(theta,wires=0)
    qml.RZ(phi,wires=0)
    return qml.expval(qml.PauliZ(0))

@qml.qnode(qpu,interface='torch')
def dummy_circ():
    return qml.expval(qml.PauliZ(0))

In [12]:
dummy_circ()

tensor(0.9600, dtype=torch.float64)

In [13]:
def cost(phi,theta):
    return torch.abs(circuit(phi,theta)-target)**2

In [14]:
opt=torch.optim.Adam([phi,theta],lr=0.1)

In [16]:
print ('theta {}, phi {}'.format(theta,phi))

theta 9.999999747378752e-06, phi 0.0


In [17]:
cost(phi,theta)

tensor(3.6864, dtype=torch.float64, grad_fn=<PowBackward0>)

In [18]:
for i in range(200):
    print ('theta {}, phi {}'.format(theta,phi))
    opt.zero_grad()
    loss=cost(phi,theta)
    loss.backward()
    opt.step()

theta 9.999999747378752e-06, phi 0.0
theta 0.1000099927186966, phi 0.09999996423721313
theta 0.19982731342315674, phi 0.05113063380122185
theta 0.29990488290786743, phi -0.019771356135606766
theta 0.39858749508857727, phi -0.08966183662414551
theta 0.49705952405929565, phi -0.17006289958953857
theta 0.5957006216049194, phi -0.25682616233825684
theta 0.6948927044868469, phi -0.34795939922332764
theta 0.7944105863571167, phi -0.4412258267402649
theta 0.8951951265335083, phi -0.4990694522857666
theta 0.9972129464149475, phi -0.5308352112770081
theta 1.100498080253601, phi -0.552353024482727
theta 1.204092264175415, phi -0.5850476026535034
theta 1.3081650733947754, phi -0.6315405964851379
theta 1.4127613306045532, phi -0.6483408212661743
theta 1.517758846282959, phi -0.6510787606239319
theta 1.6224817037582397, phi -0.6609950065612793
theta 1.7272565364837646, phi -0.6699919700622559
theta 1.830336570739746, phi -0.6811250448226929
theta 1.9319616556167603, phi -0.6801068186759949
theta 2.

RequestsApiError: 504 Server Error: Gateway Time-out for url: https://api.quantum-computing.ibm.com/api/Network/ibm-q/Groups/open/Projects/main/Jobs?access_token=...